In [ ]:
flightStatsTZ/GJS4556
/ajms/GJS4556

UA
421
622
444
447

AA
7284
6122
611

DL
499
498
511
512
514



In [ ]:
from routes.flight_aggregator_routes import flight_aware_w_auth


a = await flight_aware_w_auth('UAL421')


In [ ]:
a

In [ ]:
from core.api.flightStats import FlightStatsScraper
from core.api.flightStats import FlightStatsExtractor


flightID = 'AA622'
fss = FlightStatsScraper()
soup = fss.scrape(flightID=flightID, return_bs4=True)


fse = FlightStatsExtractor()
fs_data = fse.ticket_card(soup_fs=soup)
fs_data
# fse.ticket_card_extracts(a)

In [ ]:
from services.flight_aggregator_service import flight_stats_url_service


a = await flight_stats_url_service('AA622')
a

In [ ]:
from services.flight_aggregator_service import Pull_flight_info
flightID = 'UAL4310'
flt_info = Pull_flight_info()
# TODO search suggestions: account for this in major icao_to_iata match
        # For raw submits IATA goes to flightstats first. 
            # UA4433 search digits related to United and united regionals - SKW, RPA, GJS, mesa or commuteair. compare flightstats airport returns with jms to determine
            # What is the fallback?

fs_departure_arr_time_zone = flt_info.flightStats_data_frontend_format(flightID)


In [ ]:
fs_sop

In [ ]:
multi_flight = soup.select('[class*="past-upcoming-flights__TextHelper"]')
flights_data = []
current_date = None
current_flight = []

for element in multi_flight:
    classes = element.get('class', [])
    text = element.get_text(strip=True)
    
    if 'bTIClP' in classes and text.startswith('Flights for'):
        current_date = text.replace('Flights for ', '')
        continue
    
    if 'HeaderText' in ' '.join(classes):
        continue
    
    if current_date:
        if 'cUykul' in classes or 'cjETuy' in classes or 'diRMZq' in classes:
            current_flight.append(text)
            
            # When we have 6 items (dep_time, origin, origin_name, dest, dest_name, arr_time)
            if len(current_flight) == 6:
                flights_data.append({
                    'date': current_date,
                    'departure_time': current_flight[0],
                    'origin': current_flight[1],
                    'origin_name': current_flight[2],
                    'destination': current_flight[3],
                    'destination_name': current_flight[4],
                    'arrival_time': current_flight[5]
                })
                current_flight = []

In [ ]:
each_multi

In [ ]:
from datetime import datetime


multi_date_format = '%A, %d-%b-%Y'
solo_date_format = '%d-%b-%Y'

dep_extracts = fs_sop['fsDeparture']
departure_date = datetime.strptime(dep_extracts['ScheduledDate'], solo_date_format)
# Initialize a dictionary to store the count of dates
date_count = {}

multi_extracts = []
for each_multi in flights_data:
    each_date = datetime.strptime(each_multi['date'], multi_date_format)
    if each_date== departure_date:
        multi_extracts.append(each_multi)
        # print(each_date,departure_date)
if len(multi_extracts) >1:
    print(multi_extracts)

In [ ]:
import requests
from bs4 import BeautifulSoup
class EDCT_LookUp:
    def __init__(self):
        # URL of the EDCT lookup page
        self.url = "https://www.fly.faa.gov/edct/showEDCT"
        # self.url = "https://www.fly.faa.gov/edct/"
        
    def extract_edct(self, call_sign: str, origin: str, destination: str):
        # Form data
        data = {
            "callsign": call_sign.upper(),  # Convert to uppercase to match form behavior
            "dept": origin.upper(),
            "arr": destination.upper(),
        }

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Referer': self.url,  # Important for many forms
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
        }
        # Send POST request
        # response = requests.post(self.url, data=data, headers=headers)
        response = requests.post(self.url, data=data)
        edct_collective = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the results table (same as Selenium's border=1 table)
            # print(soup)
            table = soup.find('table', {'border': '1'})
            
            if table:
                # Process all rows except header (same as rows[1:] in Selenium)
                rows = table.find_all('tr')[1:]
                
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 4:  # Ensure we have enough columns
                        edct = cols[0].get_text(strip=True)
                        filed_departure_time = cols[1].get_text(strip=True)
                        control_element = cols[2].get_text(strip=True)
                        flight_cancelled = cols[3].get_text(strip=True)
                        
                        if edct != '--':
                            edct_collective.append({
                                "filedDepartureTime": filed_departure_time,
                                "edct": edct,
                                "controlElement": control_element,
                                "flightCancelled": flight_cancelled
                            })
            
            return edct_collective
        else:
            print(f"Request failed with status {response.status_code}")
            return None

# Example usage:
edct = EDCT_LookUp()
results = edct.extract_edct("GJS4413", "DCA", "EWR")
print(results)
